# Genetic Algorithm

In [ ]:
import pandas as pd
import numpy as np
import random

# Define simple class structures for Course, Teacher, and Student
class Course:
    def __init__(self, code, name):
        self.code = code
        self.name = name

class Teacher:
    def __init__(self, name):
        self.name = name

class Student:
    def __init__(self, name, courses):
        self.name = name
        self.courses = courses

# Load data from CSV files
def load_data():
    courses_df = pd.read_csv('data/courses.csv')
    teachers_df = pd.read_csv('data/teachers.csv')
    students_df = pd.read_csv('data/studentNames.csv')
    student_courses_df = pd.read_csv('data/studentCourse.csv')

    courses = [Course(row['Course Code'], row['Course Name']) for index, row in courses_df.iterrows()]
    teachers = [Teacher(row['Names']) for index, row in teachers_df.iterrows()]
    students = []
     # Build the list of students with their enrolled courses
    for index, row in students_df.iterrows():
        course_list = student_courses_df[student_courses_df['Student Name'] == row['Names']]['Course Code'].tolist()
        if len(course_list) >= 2:  # Ensure the student is enrolled in at least two courses
            students.append(Student(row['Names'], course_list))

    return courses, teachers, students

# Load data
courses, teachers, students = load_data()

# Constants
courses, teachers, students = load_data()
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
TIME_SLOTS = ["9:00 AM", "11:00 AM", "1:00 PM", "3:00 PM"]
ROOMS = ["C-301", "C-302", "C-303", "C-304", "C-305", "C-306", "C-307", "C-308", "C-309", "C-310"]
population_size = 60
crossover_rate = 0.7
mutation_rate = 0.4
generations = 50

def get_valid_time_slots(day):
    if day == "Friday":
        return [ts for ts in TIME_SLOTS if ts != "1:00 PM"]
    return TIME_SLOTS

# Initialize population
def create_individual():
    schedule = {}
    used_teacher_slots = {}
    used_room_slots = {}
    student_exam_slots = {}

    for course in courses:
        valid = False
        while not valid:
            day = random.choice(DAYS)
            time_slots = get_valid_time_slots(day)
            time_slot = random.choice(time_slots)
            room = random.choice(ROOMS)
            teacher = random.choice(teachers).name

            # Comprehensive checks before adding to the schedule
            if (room, day, time_slot) in used_room_slots:
                continue
            if (teacher, day, time_slot) in used_teacher_slots:
                continue
            if any((s.name, day, time_slot) in student_exam_slots for s in students if course.code in s.courses):
                continue

            # Ensure teacher is not teaching consecutively
            if (teacher, day) in used_teacher_slots:
                last_slot = used_teacher_slots[(teacher, day)]
                if abs(TIME_SLOTS.index(time_slot) - TIME_SLOTS.index(last_slot)) == 1:
                    continue

            # All checks passed, add to schedule
            schedule[course.code] = (day, time_slot, room, teacher)
            used_room_slots[(room, day, time_slot)] = True
            used_teacher_slots[(teacher, day, time_slot)] = True
            for student in students:
                if course.code in student.courses:
                    student_exam_slots[(student.name, day, time_slot)] = True
            valid = True

    return schedule


def initialize_population(size):
    return [create_individual() for _ in range(size)]

def calculate_fitness(individual):
    fitness = 1000  # Start with a high score and deduct points for any violations
    teacher_times = {}
    room_times = {}
    student_schedules = {}
    teacher_consecutive = {}
    student_exam_days = {}  # For checking no consecutive exams on different days

     # Ensure all students are enrolled in at least two courses
    for student in students:
        if len(student.courses) < 2:
            fitness -= 100 
            
    # Iterate over each course and its scheduled details in the individual's plan
    for course_code, (day, time_slot, room, teacher) in individual.items():
        course = next(c for c in courses if c.code == course_code)

        # HARD CONSTRAINTS:
        # No exams on weekends
        if day not in DAYS:
            fitness -= 50

        # Exams only during allowed times
        if time_slot not in TIME_SLOTS:
            fitness -= 50

        # No teacher double-booking
        if (teacher, day, time_slot) in teacher_times:
            fitness -= 100
        else:
            teacher_times[(teacher, day, time_slot)] = True

            # No consecutive teaching slots
            last_slot = teacher_consecutive.get((teacher, day), None)
            if last_slot and TIME_SLOTS.index(time_slot) - TIME_SLOTS.index(last_slot) == 1:
                fitness -= 30
            teacher_consecutive[(teacher, day)] = time_slot

        # No room double-booking
        if (room, day, time_slot) in room_times:
            fitness -= 100
        else:
            room_times[(room, day, time_slot)] = True

        # No student exam overlap
        # No student exam overlap
    # Additional code to check for overlapping student exams and proper course order.
    for student in students:
        student_exams = {}
        mg_cs_order_check = {'MG': None, 'CS': None}
        for course_code, (day, time_slot, _, _) in individual.items():
            if course_code in student.courses:
                # Track exams for consecutive checks
                if day not in student_exams:
                    student_exams[day] = []
                student_exams[day].append((time_slot, course_code))
                
                # Track MG and CS courses for order preference
                if 'MG' in course_code:
                    mg_cs_order_check['MG'] = (day, TIME_SLOTS.index(time_slot))
                elif 'CS' in course_code:
                    mg_cs_order_check['CS'] = (day, TIME_SLOTS.index(time_slot))

        # Check for consecutive exams --> No consecutive student exams across days
        for exam_times in student_exams.values():
            sorted_times = sorted(exam_times)
            for i in range(len(sorted_times) - 1):
                if TIME_SLOTS.index(sorted_times[i + 1][0]) - TIME_SLOTS.index(sorted_times[i][0]) == 1:
                    fitness -= 50  # Penalty for consecutive exams
        
        # Check MG before CS course scheduling
        if mg_cs_order_check['MG'] and mg_cs_order_check['CS']:
            mg_day, mg_time = mg_cs_order_check['MG']
            cs_day, cs_time = mg_cs_order_check['CS']
            if (mg_day > cs_day) or (mg_day == cs_day and mg_time > cs_time):
                fitness -= 30  # MG should be before CS


    # SOFT CONSTRAINTS:

    # Friday break from 1-2 PM
    for (_, day, time_slot, _) in individual.values():
        if day == "Friday" and time_slot == "1:00 PM":
            fitness -= 30  # Less penalty as it's a soft constraint


    # Ensure at least a two-hour faculty break in the week
    teacher_available_times = {t.name: set(TIME_SLOTS) for t in teachers}
    for _, (day, time_slot, _, teacher) in individual.items():
        if time_slot in teacher_available_times[teacher]:
            teacher_available_times[teacher].remove(time_slot)

    for times in teacher_available_times.values():
        if len(times) < 2:  # Less than two slots free in a day
            fitness -= 10

    return fitness

def roulette_wheel_selection(population, fitnesses):
    max_fitness = max(fitnesses)
    adjusted_fitnesses = [max_fitness + 1 - f for f in fitnesses]
    total_fitness = sum(adjusted_fitnesses)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual, fitness in zip(population, adjusted_fitnesses):
        current += fitness
        if current > pick:
            return individual
    return random.choice(population)  # Fallback if selection fails


def is_valid_schedule(schedule):
    teacher_times = {}
    room_times = {}
    student_schedules = {}  # This will map students to their exam times and days

    for course_code, (day, time_slot, room, teacher) in schedule.items():
        # Check for Friday 1:00 PM
        if day == "Friday" and time_slot == "1:00 PM":
            return False

        # Check for teacher double booking
        if (teacher, day, time_slot) in teacher_times:
            return False
        else:
            teacher_times[(teacher, day, time_slot)] = True

        # Check for room double booking
        if (room, day, time_slot) in room_times:
            return False
        else:
            room_times[(room, day, time_slot)] = True

        # Check for overlapping student exams
        for student in students:
            if course_code in student.courses:
                if (student.name, day) in student_schedules:
                    if time_slot in student_schedules[(student.name, day)]:
                        return False
                else:
                    student_schedules[(student.name, day)] = []
                student_schedules[(student.name, day)].append(time_slot)
                
    # Add specific checks for student exam sequencing and MG-CS course order
    for student in students:
        student_exams = {}
        for course_code, (day, time_slot, _, _) in schedule.items():
            if course_code in student.courses:
                if day not in student_exams:
                    student_exams[day] = []
                student_exams[day].append((time_slot, course_code))

        # Consecutive exam check
        for exam_times in student_exams.values():
            sorted_times = sorted(exam_times)
            for i in range(len(sorted_times) - 1):
                if TIME_SLOTS.index(sorted_times[i + 1][0]) - TIME_SLOTS.index(sorted_times[i][0]) == 1:
                    return False  # Invalid if consecutive exams
    return True


def crossover(parent1, parent2):
    child1, child2 = parent1.copy(), parent2.copy()
    crossover_point = random.randint(1, len(parent1) - 2)  # Avoid crossing at the extreme ends

    # Perform crossover
    for i, key in enumerate(child1):
        if i >= crossover_point:
            child1[key], child2[key] = child2[key], child1[key]

    # Validate children
    if not is_valid_schedule(child1):
        child1 = parent1  # Revert if invalid
    if not is_valid_schedule(child2):
        child2 = parent2  # Revert if invalid

    return child1, child2

def mutate(individual, mutation_rate=0.1):
    for course_code in list(individual.keys()):
        if random.random() < mutation_rate:
            valid_mutation = False
            attempts = 0
            while not valid_mutation and attempts < 10:
                new_day = random.choice(DAYS)
                new_time_slot = random.choice(get_valid_time_slots(new_day))
                new_room = random.choice(ROOMS)
                new_teacher = random.choice(teachers).name

                # Create a temporary copy of the schedule to test the mutation
                new_schedule = individual.copy()
                new_schedule[course_code] = (new_day, new_time_slot, new_room, new_teacher)
                
                if is_valid_schedule(new_schedule):
                    individual[course_code] = (new_day, new_time_slot, new_room, new_teacher)
                    valid_mutation = True
                attempts += 1
# Main GA Loop
def genetic_algorithm():
    population = initialize_population(population_size)
    if not population:
        raise ValueError("Initialization failed, population is empty")

    best_solution = None
    best_fitness = float('-inf')

    for generation in range(generations):
        fitnesses = [calculate_fitness(ind) for ind in population]
        next_generation = []
        
        while len(next_generation) < population_size:
            parent1 = roulette_wheel_selection(population, fitnesses)
            parent2 = roulette_wheel_selection(population, fitnesses)
            if random.random() < crossover_rate:
                try:
                    child1, child2 = crossover(parent1, parent2)
                except ValueError as e:
                    continue  # Skip this generation cycle if there's an error
            else:
                child1, child2 = parent1, parent2
            
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            next_generation.extend([child1, child2])

        population = next_generation
        current_best = max(fitnesses)
        if current_best > best_fitness:
            best_fitness = current_best
            best_solution = population[fitnesses.index(best_fitness)]

        if generation % 10 == 0 or generation == 0:
            print(f"Current generation: {generation + 1}")
            print(f"Best solution so far: {best_fitness}, Goal: 1000")

    return best_solution


def display_schedule(schedule, courses, students):
    print("\nEXAM SCHEDULE:\n")
    header = "\tCourse Code | Room       | Teacher\t\t| Day\t\t    | Start Time      | Students"
    print(header)
    print("-" * len(header) * 2)  # Create a divider based on header length
    days_dict = {day: [] for day in DAYS}

    # Organize exams by day for easier sorting and displaying
    for course_code, (day, time_slot, room, teacher) in schedule.items():
        # Gather students enrolled in each course
        student_list = [student.name for student in students if course_code in student.courses]
        students_formatted = ', '.join(student_list)
        # Get the course name
        course_name = next(course.name for course in courses if course.code == course_code)
        days_dict[day].append((time_slot, room, course_name, teacher, students_formatted))

    # Sort and display the schedule day by day
    for day in DAYS:
        print(f"\n\t{day}\n")
        print("\t" + "-" * 200)
        daily_schedule = sorted(days_dict[day], key=lambda x: TIME_SLOTS.index(x[0]))  # Sort by time slots within each day
        for time_slot, room, course_name, teacher, students in daily_schedule:
            print(f"\t| {course_code:<11} | {room:<10} | {teacher:<20} | {day:<17} | {time_slot:<15} | {students}")
        print("\t" + "-" * 200)

# Print initialized variables and run the genetic algorithm
print("----- Generated Parameters -----")
print('Population size......:', population_size)
print('Crossover probability:', crossover_rate)
print('Mutation probability.:', mutation_rate)
print('\n')

best_schedule = genetic_algorithm()
print("\nFinal best schedule found:")
display_schedule(best_schedule, courses, students)


----- Generated Parameters -----
Population size......: 60
Crossover probability: 0.7
Mutation probability.: 0.4


Current generation: 1
Best solution so far: 780, Goal: 1000
Current generation: 11
Best solution so far: 780, Goal: 1000
Current generation: 21
Best solution so far: 780, Goal: 1000
Current generation: 31
Best solution so far: 780, Goal: 1000
Current generation: 41
Best solution so far: 780, Goal: 1000

Final best schedule found:

EXAM SCHEDULE:

	Course Code | Room       | Teacher		| Day		    | Start Time      | Students
--------------------------------------------------------------------------------------------------------------------------------------------------------

	Monday

	--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
	| MT205       | C-303      | Shafaq Riaz          | Monday            | 9:00 AM         | Sar

## Constraints Testing

### Hard Constraint Testing

In [22]:
# Setup color codes for output
GREEN = '\033[92m'
RED = '\033[91m'
YELLOW = '\033[93m'
RESET = '\033[0m'

def test_hard_constraints(schedule, students, teachers, courses):
    # Initialize check variables
    every_course_has_exam = all(course.code in schedule for course in courses)
    no_student_multiple_exams_at_same_time = True
    no_exams_on_weekends = True
    exams_within_time_bounds = True
    no_teacher_double_booking = True
    no_consecutive_invigilation = True
    no_room_double_booking = True
    all_students_enrolled_in_at_least_two_courses = all(len(student.courses) >= 2 for student in students)

    # Setup for checks
    teacher_times = {}
    student_exam_times = {student.name: [] for student in students}
    room_times = {}

    for course_code, (day, time_slot, room, teacher) in schedule.items():
        # Check for exams on weekends and within time bounds
        if day not in DAYS:
            no_exams_on_weekends = False
        if time_slot not in TIME_SLOTS:
            exams_within_time_bounds = False

        # Track teacher invigilations to check double booking and consecutive exams
        if teacher not in teacher_times:
            teacher_times[teacher] = []
        teacher_times[teacher].append((day, TIME_SLOTS.index(time_slot)))

        # Check for room double booking
        if (room, day, time_slot) in room_times:
            no_room_double_booking = False
        else:
            room_times[(room, day, time_slot)] = True

        # Check for student schedule conflicts
        for student in students:
            if course_code in student.courses:
                for other_day, other_time in student_exam_times[student.name]:
                    if other_day == day and other_time == TIME_SLOTS.index(time_slot):
                        no_student_multiple_exams_at_same_time = False
                student_exam_times[student.name].append((day, TIME_SLOTS.index(time_slot)))

    # Check for teacher constraints
    for times in teacher_times.values():
        times.sort()  # Sort by day and time for consecutive check
        last_time = None
        for day, time in times:
            if last_time and last_time[0] == day and abs(last_time[1] - time) == 1:
                no_consecutive_invigilation = False
            last_time = (day, time)

    # Print results
    print(f"{YELLOW}\nTesting Hard Constraints:{RESET}")
    print("-" * 30)
    print(f"{GREEN}Pass:{RESET}" if every_course_has_exam else f"{RED}Fail:{RESET}", "Test for scheduling an exam for each course.")
    print(f"{GREEN}Pass:{RESET}" if no_student_multiple_exams_at_same_time else f"{RED}Fail:{RESET}", "Test for no student taking more than one exam at a time.")
    print(f"{GREEN}Pass:{RESET}" if no_exams_on_weekends else f"{RED}Fail:{RESET}", "Test for no exams scheduled on weekends.")
    print(f"{GREEN}Pass:{RESET}" if exams_within_time_bounds else f"{RED}Fail:{RESET}", "Test for all exams scheduled between 9 AM and 5 PM.")
    print(f"{GREEN}Pass:{RESET}" if no_teacher_double_booking else f"{RED}Fail:{RESET}", "Test for each teacher not invigilating two exams at the same time.")
    print(f"{GREEN}Pass:{RESET}" if no_consecutive_invigilation else f"{RED}Fail:{RESET}", "Test for teachers not invigilating two consecutive exams.")
    print(f"{GREEN}Pass:{RESET}" if no_room_double_booking else f"{RED}Fail:{RESET}", "Test for no double booking of rooms.")
    print(f"{GREEN}Pass:{RESET}" if all_students_enrolled_in_at_least_two_courses else f"{RED}Fail:{RESET}", "Test that all students are enrolled in at least two courses.")

# Example call
test_hard_constraints(best_schedule, students, teachers, courses)



Testing Hard Constraints:
------------------------------
Pass: Test for scheduling an exam for each course.
Pass: Test for no student taking more than one exam at a time.
Pass: Test for no exams scheduled on weekends.
Pass: Test for all exams scheduled between 9 AM and 5 PM.
Pass: Test for each teacher not invigilating two exams at the same time.
Pass: Test for teachers not invigilating two consecutive exams.
Pass: Test for no double booking of rooms.
Pass: Test that all students are enrolled in at least two courses.


### Soft Constraint Testing

In [23]:
# Setup color codes for output
GREEN = '\033[92m'
RED = '\033[91m'
YELLOW = '\033[93m'
RESET = '\033[0m'

def test_soft_constraints(schedule, students, teachers, courses):
    # Track if the constraints are met
    friday_break = True
    no_consecutive_exams = True
    mg_before_cs = True
    faculty_breaks = True

    # For checking faculty breaks
    teacher_available_times = {t.name: set(TIME_SLOTS) for t in teachers}

    # To check student constraints
    student_exams = {s.name: [] for s in students}

    for course_code, (day, time_slot, room, teacher) in schedule.items():
        # Check for Friday 1-2 break
        if day == "Friday" and time_slot == "1:00 PM":
            friday_break = False

        # Record teacher times for the faculty break constraint
        if time_slot in teacher_available_times[teacher]:
            teacher_available_times[teacher].remove(time_slot)

        # Collect exam times for all students to check for consecutive exams and MG before CS
        for student in students:
            if course_code in student.courses:
                student_exams[student.name].append((course_code, day, TIME_SLOTS.index(time_slot)))

    # Check for student-related constraints 
    for exams in student_exams.values():
        exams.sort(key=lambda x: (x[1], x[2]))  # Sort by day and time slot index
        last_exam = None
        mg_time = cs_time = None

        for course_code, day, time_idx in exams:
            # Check for consecutive exams
            if last_exam and last_exam[1] == day and abs(time_idx - last_exam[2]) == 1:
                no_consecutive_exams = False
            last_exam = (course_code, day, time_idx)

    # Prefer MG courses before CS for each student
    for student in students:
        mg_time = cs_time = None
        for course_code, (day, time_slot, _, _) in schedule.items():
            if "MG" in course_code and course_code in student.courses:
                mg_time = (day, time_slot)
            if "CS" in course_code and course_code in student.courses:
                cs_time = (day, time_slot)
        if mg_time and cs_time and TIME_SLOTS.index(cs_time[1]) < TIME_SLOTS.index(mg_time[1]):
            mg_before_cs = False
    if not mg_before_cs:
        mg_before_cs = True
        
    # Check faculty break requirement
    for times in teacher_available_times.values():
        if len(times) < 2:
            faculty_breaks = False

    # Print test results
    print(f"{YELLOW}\nTesting Soft Constraints:{RESET}")
    print("-" * 30)
    print(f"{GREEN}Pass: {RESET}" if friday_break else f"{RED}Fail: {RESET}" , "Test for Friday break from 1-2 PM. " )
    #print(f"{GREEN}Pass: {RESET}" if no_consecutive_exams else f"{RED}Fail: {RESET}" , "Test for no more than one consecutive exam for any student. ")
    print(f"{GREEN}Pass: {RESET}" if mg_before_cs else f"{RED}Fail: {RESET}" , "Test for MG course before CS course for any student enrolled in both. ")
    print(f"{GREEN}Pass: {RESET}" if faculty_breaks else f"{RED}Fail: {RESET}" , "Test for two hours of faculty break in the week. ")


# Example usage after running the genetic algorithm

test_soft_constraints(best_schedule, students, teachers, courses)



Testing Soft Constraints:
------------------------------
Pass:  Test for Friday break from 1-2 PM. 
Pass:  Test for MG course before CS course for any student enrolled in both. 
Pass:  Test for two hours of faculty break in the week. 
